In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from rushhour_objects import *

## Simple examples using 'Car'

In [6]:
# Car placed horizontally
car = Car(color="green",position=[0,0],length=3,orientation="h")
print("\nOriginal       : %s"%car)

# Displacement along the 'free' axis
car += 1
print("Displacement +1: %s"%car)
car -= 1
print("Displacement -1: %s"%car)

# Car placed vertically
car = Car(color="green",position=[0,0],length=3,orientation="v")
print("\nOriginal       : %s"%car)

# Displacement along the 'free' axis
car += 1
print("Displacement +1: %s"%car)
car -= 1
print("Displacement -1: %s"%car)


# Assume that a car is placed in a board of dimension 6x6
print("\nChecking the acceptance of moves")
print("Original       : %s"%car)
for i in range(6):
    if car.can_move(+1,6): 
        car += 1
        print("- Move accepted: %s"%car)
    else:
        print("- Move denied  : %s"%car)
        
# Moving back again (lower limit is 0)
for i in range(6):
    if car.can_move(-1,0): 
        car -= 1
        print("- Move accepted: %s"%car)
    else:
        print("- Move denied  : %s"%car)


Original       : Instance of Car(color=green,position=(0, 0),length=3,orientation=h)
Displacement +1: Instance of Car(color=green,position=(0, 1),length=3,orientation=h)
Displacement -1: Instance of Car(color=green,position=(0, 0),length=3,orientation=h)

Original       : Instance of Car(color=green,position=(0, 0),length=3,orientation=v)
Displacement +1: Instance of Car(color=green,position=(1, 0),length=3,orientation=v)
Displacement -1: Instance of Car(color=green,position=(0, 0),length=3,orientation=v)

Checking the acceptance of moves
Original       : Instance of Car(color=green,position=(0, 0),length=3,orientation=v)
- Move accepted: Instance of Car(color=green,position=(1, 0),length=3,orientation=v)
- Move accepted: Instance of Car(color=green,position=(2, 0),length=3,orientation=v)
- Move accepted: Instance of Car(color=green,position=(3, 0),length=3,orientation=v)
- Move denied  : Instance of Car(color=green,position=(3, 0),length=3,orientation=v)
- Move denied  : Instance of 

### Dealing with the position of the car in terms of slices

In [ ]:
# Create a board
bb = np.zeros([6,6])
print("Original\n%s"%bb)

# Three cars in different positions 
car1 = Car(color="green",position=[0,0],length=3,orientation="h")
car2 = Car(color="green",position=[1,1],length=3,orientation="h")
car3 = Car(color="green",position=[3,5],length=3,orientation="v")
bb[car1.npindices] = 1
bb[car2.npindices] = 1
bb[car3.npindices] = 1
print("Placement\n%s"%bb)



## Board manipulations

In [ ]:
# Create the board
board = Board(5,2)

# Insert some cars (more cross validation needed probably) 
board.insert_car("green",[0,0],3,"v")
board.insert_car("red",[0,2],3,"v")

board.insert_car("orange",[4,1],3,"h")

# Always a good practice to check the following line 
occupied_positions = 3 + 3 + 3
if occupied_positions != board.occupied_places:
    raise ValueError("Something went wrong in insersion")
print("--> Simple printout")
print(board)
print("\nRendered view of the board\n")
board.render()

### Let's check first all the connected states from the above configuration

In [ ]:
for index,state in enumerate(board.connected_states):
    Board.from_state(state).render(title="Connection-%s with hash = %s"%(index,hash(state)),padding="  ") 

# But (of course) no change in the original 
board.render(title="Parent board-view with hash = %s"%hash(board.get_state()),padding="  ")

## Ways to move manually a car that is inserted to the board

### 1. Direct access to the specific car and usage of operations += and -=   

Attention: In this case you change the position of the car BUT you don't update the view of the board
and you do not check for conflicts

In [ ]:
board.cars["green"] += 1 # Car moved one position

In [ ]:
board.render("The render reconstructs the board from the cars!!")

but the original view is not updated!!

In [ ]:
board

### This is a design pattern for protection. If you want to update the view you have to ask for it!

In [ ]:
board.update_view(); board

### 2. Access to the 'car' within the 'board' overloads (recommended)

In this case, you will update the position of the car ONLY if no conflicts arise either by occupied   
positions from other cars or from the boundaries of the board. If no conflicts occure the view of the   
board will be updated automatically.

In [ ]:
board.render(title="Current-point")

In [ ]:
board["orange"] -= 1 # Move to the left
board.render(title="everything fine: moving was accepted")

### trying again to move on the left (must be denied) 

In [ ]:
board["orange"] -= 1

# Note that we got but to the last step
board.render()

### let's come back to original position

In [ ]:
board["orange"] += 1

# Note that we got but to the last step
board.render()

### Custom solution - For checking against the old implementation

In [1]:
from collections import deque

In [2]:
class RushHour(object):
    """
    Class to play the game. 
    
    Arguments:
    ----------
    - initial: Board or tuple(state)
    
    Keyword arguments:
    ------------------
    - target: string, the target car (default is 'red') 
    
    """
    def __init__(self,initial,**kwargs):
        if isinstance(initial,Board):
            self.initial = Board.from_state(initial.get_state())
        elif isinstance(initial,tuple):
            self.initial = Board.from_state(initial)
        else:
            raise ValueError("RushHour initialized expects 'Board'/'tuple'") 
        
        # Cross validate the existence of the 'car' to move in the exit
        # and the corresponding 'target' position to be reached
        self.target_car = kwargs.get("target","red") 
        if self.target_car not in self.initial.cars:
        #if not self.initial.cars.has_key(self.target_car):
            raise ValueError("Target 'car' not found in the board") 
        else:
            if self.initial[self.target_car].position[0]!=self.initial.exitrow:
                raise ValueError("Target 'car' not placed in the 'exitrow'")
            else:
                targetcol = self.initial.view.shape[0]-self.initial[self.target_car].length 
                self.target = (self.initial.exitrow,targetcol)
        
            
    def check_target(self,node):
        """Routine to check if the target is reached"""
        return node.cars[self.target_car].position == self.target 
    
    def solve(self,max_iterations,use_last=False,printouts=100):
        """Implementation of the breadth-first-graph-search"""
        # Possibility to continue a calculation 
        if use_last and hasattr(self,"last"):
            print("Continuing the solver from the last parent")
            frontier = deque([Board.from_state(self.last.get_state())])
        else:
            if use_last: print("No previous solution. Falling back to initial")
            frontier = deque([Board.from_state(self.initial.get_state())]) 
        
        # Early check for available solution 
        if self.check_target(frontier[0]):
            print("Solution is already available")
            self.last = frontier[0]
            return True
            
        # The actual implementation (recursive method) 
        explored = set() ;loops    = 0
        while frontier:
            parent = frontier.popleft() 
            explored.add(parent.get_state())
            
            for child in parent.connected_states:
                node = Board.from_state(child)
                if node not in frontier and child not in explored:
                    if self.check_target(node):
                        self.last = node
                        print("Solution reached in %s"%loops)
                        self.last.render("Problem solved")
                        return True
                    frontier.append(node)
            
            loops += 1
            if loops%printouts == 0:
                for index,parent in enumerate(frontier):
                    parent.render("Frontier-%s board at %s loops"%(index,loops))
            if loops == max_iterations: break
        
        print("Unable to find solution in %s loops"%max_iterations)
        self.last = parent
        return False 

In [3]:
# Create the board
board = Board(5,2)

# Insert some cars (more cross validation needed probably) 
board.insert_car("green",[1,2],2,"v")
board.insert_car("red",[2,0],2,"h")
board.insert_car("orange",[0,1],3,"h")
board.insert_car("blue",[3,2],2,"h")

board.render()

NameError: name 'Board' is not defined

In [ ]:
Game = RushHour(board)
Game.solve(max_iterations=300,use_last=False,printouts=30)

In [ ]:
Game.initial.render("Initial board")
Game.last.render("Last board")